In [1]:
# data management
import numpy as np                   # for linear algebra
import pandas as pd                  # for tabular data manipulation and processing
import category_encoders             # for categorical encoding

# machine learning
import sklearn                       # for data prep and classical ML

# data visualization and graphics
import matplotlib.pyplot as plt      # for visualization fundamentals
import seaborn as sns                # for pretty visualizations
sns.set_palette("magma")

# misc
import math                          # for calculation
import sys                           # for system manipulation
import os                            # for file manipulation

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [3]:
# Load Dataset

file_path = "/Users/Sebastiano/data/Clinical_MRI.xlsx"
db = pd.read_excel(file_path)
pd.set_option('display.max_columns', None)

print("N° of patients: {}".format(len(db)))
print("N° of columns: {}".format(db.shape[1]))
db.head()

N° of patients: 27
N° of columns: 969


,Patient,Date of Birth,Gender,Education,Disease duration (months),Age,SLEDAI-2k (at the time of NP event),PGA (at the time of fMRI),SLICC-DI (at the time of NP event),anti-dsDNA Titre (0=absent; 1=present) ),anti-dsDNA Titre (insert NV here <7 ),Anti-Ro-SSA,Anti-La-SSB,Anti-RNP,anti-Sm,C3 (mgdl),C4 (mgdl),aPL syndrome,LAC,aCL IgG,aCL IgM,aB2GPI IgG,aB2GPI IgM,aPL+,No Treatment,Antiplatelet,Anticoagulant,Prednisone (mg equivalent),Antimalarial,Immunosuppressant,Biologic,AnAb,Anti-Rib-P,Anti-DWEYS,NP-SLE,Event,Livedo reticularis,Cutaneous vasculitis,Hypertension,Hyperlipidaemia,current Smoking,ever smoking,Scale factor,SNR,White Matter (WM) volume cm3,White Matter (WM) volume %,Normal Appearing White Matter volume cm3,Normal Appearing White Matter volume %,Abnormal Appearing White Matter volume cm3,Abnormal Appearing White Matter volume %,Grey Matter (GM) volume cm3,Grey Matter (GM) volume %,Subcortical Grey Matter volume cm3,Subcortical Grey Matter volume %,Cortical Grey Matter volume cm3,Cortical Grey Matter volume %,Cerebellar Grey Matter volume cm3,Cerebellar Grey Matter volume %,Cerebro Spinal Fluid (CSF) volume cm3,Cerebro Spinal Fluid (CSF) volume %,Brain (WM+GM) volume cm3,Brain (WM+GM) volume %,Intracranial Cavity (IC) volume cm3,Intracranial Cavity (IC) volume %,Cerebrum total volume cm3,Cerebrum total volume %,Cerebrum right volume cm3,Cerebrum right volume %,Cerebrum left volume cm3,Cerebrum left volume %,Cerebrum volume asymmetry,Cerebrum WM total volume cm3,Cerebrum WM total volume %,Cerebrum WM right volume cm3,Cerebrum WM right volume %,Cerebrum WM left volume cm3,Cerebrum WM left volume %,Cerebrum WM volume asymmetry,Cerebrum GM total volume cm3,Cerebrum GM total volume %,Cerebrum GM right volume cm3,Cerebrum GM right volume %,Cerebrum GM left volume cm3,Cerebrum GM left volume %,Cerebrum GM volume asymmetry,Cerebellum total volume cm3,Cerebellum total volume %,Cerebellum right volume cm3,Cerebellum right volume %,Cerebellum left volume cm3,Cerebellum left volume %,Cerebellum volume asymmetry,Cerebellum WM total volume cm3,Cerebellum WM total volume %,Cerebellum WM right volume cm3,Cerebellum WM right volume %,Cerebellum WM left volume cm3,Cerebellum WM left volume %,Cerebellum WM volume asymmetry,Cerebellum GM total volume cm3,Cerebellum GM total volume %,Cerebellum GM right volume cm3,Cerebellum GM right volume %,Cerebellum GM left volume cm3,Cerebellum GM left volume %,Cerebellum GM volume asymmetry,Vermis volume cm3,Vermis volume %,Brainstem volume cm3,Brainstem volume %,Accumbens total volume cm3,Accumbens total volume %,Accumbens right volume cm3,Accumbens right volume %,Accumbens left volume cm3,Accumbens left volume %,Accumbens volume asymmetry,Amygdala total volume cm3,Amygdala total volume %,Amygdala right volume cm3,Amygdala right volume %,Amygdala left volume cm3,Amygdala left volume %,Amygdala volume asymmetry,Basal Forebrain total volume cm3,Basal Forebrain total volume %,Basal Forebrain right volume cm3,Basal Forebrain right volume %,Basal Forebrain left volume cm3,Basal Forebrain left volume %,Basal Forebrain volume asymmetry,Caudate total volume cm3,Caudate total volume %,Caudate right volume cm3,Caudate right volume %,Caudate left volume cm3,Caudate left volume %,Caudate volume asymmetry,Hippocampus total volume cm3,Hippocampus total volume %,Hippocampus right volume cm3,Hippocampus right volume %,Hippocampus left volume cm3,Hippocampus left volume %,Hippocampus volume asymmetry,Pallidum total volume cm3,Pallidum total volume %,Pallidum right volume cm3,Pallidum right volume %,Pallidum left volume cm3,Pallidum left volume %,Pallidum volume asymmetry,Putamen total volume cm3,Putamen total volume %,Putamen right volume cm3,Putamen right volume %,Putamen left volume cm3,Putamen left volume %,Putamen volume asymmetry,Thalamus total volume cm3,Thalamus total volume %,Thalamus right volume cm3,Thalamus right volume %,Thalamus left volume cm3,Thalamus left volume %,Thalamus volume asymmetry,Ventral DC

In [4]:
# Drop unwanted columns

df = db.drop(['Patient','Date of Birth', 'Gender', 'Education', 'Age'], axis = 'columns')
# drop columns that include "%" in their name
#cols_to_drop = [col for col in df.columns if "%" in col]
#df = df.drop(columns=cols_to_drop)
print("Effective features to consider: {} ".format(len(df.columns)-1))

Effective features to consider: 963 


In [5]:
# One Hot Encoding for Scores, Antiplatelets and Coagulants, Therapy, NP Event

from sklearn.preprocessing import OneHotEncoder

categ = ['Antiplatelet', 'Anticoagulant', 'Antimalarial', 'Immunosuppressant', 'Biologic', 'Event']
ohe = OneHotEncoder(categories='auto',sparse=False)
df_enc = ohe.fit_transform(df[categ])
df_enc = pd.DataFrame(df_enc,columns=ohe.get_feature_names_out(categ))
df = pd.concat([df, df_enc], axis=1)
df = df.drop(categ, axis=1)
df.tail()

/opt/homebrew/Caskroom/miniforge/base/envs/lupus/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Disease duration (months),SLEDAI-2k (at the time of NP event),PGA (at the time of fMRI),SLICC-DI (at the time of NP event),anti-dsDNA Titre (0=absent; 1=present) ),anti-dsDNA Titre (insert NV here <7 ),Anti-Ro-SSA,Anti-La-SSB,Anti-RNP,anti-Sm,C3 (mgdl),C4 (mgdl),aPL syndrome,LAC,aCL IgG,aCL IgM,aB2GPI IgG,aB2GPI IgM,aPL+,No Treatment,Prednisone (mg equivalent),AnAb,Anti-Rib-P,Anti-DWEYS,NP-SLE,Livedo reticularis,Cutaneous vasculitis,Hypertension,Hyperlipidaemia,current Smoking,ever smoking,Scale factor,SNR,White Matter (WM) volume cm3,White Matter (WM) volume %,Normal Appearing White Matter volume cm3,Normal Appearing White Matter volume %,Abnormal Appearing White Matter volume cm3,Abnormal Appearing White Matter volume %,Grey Matter (GM) volume cm3,Grey Matter (GM) volume %,Subcortical Grey Matter volume cm3,Subcortical Grey Matter volume %,Cortical Grey Matter volume cm3,Cortical Grey Matter volume %,Cerebellar Grey Matter volume cm3,Cerebellar Grey Matter volume %,Cerebro Spinal Fluid (CSF) volume cm3,Cerebro Spinal Fluid (CSF) volume %,Brain (WM+GM) volume cm3,Brain (WM+GM) volume %,Intracranial Cavity (IC) volume cm3,Intracranial Cavity (IC) volume %,Cerebrum total volume cm3,Cerebrum total volume %,Cerebrum right volume cm3,Cerebrum right volume %,Cerebrum left volume cm3,Cerebrum left volume %,Cerebrum volume asymmetry,Cerebrum WM total volume cm3,Cerebrum WM total volume %,Cerebrum WM right volume cm3,Cerebrum WM right volume %,Cerebrum WM left volume cm3,Cerebrum WM left volume %,Cerebrum WM volume asymmetry,Cerebrum GM total volume cm3,Cerebrum GM total volume %,Cerebrum GM right volume cm3,Cerebrum GM right volume %,Cerebrum GM left volume cm3,Cerebrum GM left volume %,Cerebrum GM volume asymmetry,Cerebellum total volume cm3,Cerebellum total volume %,Cerebellum right volume cm3,Cerebellum right volume %,Cerebellum left volume cm3,Cerebellum left volume %,Cerebellum volume asymmetry,Cerebellum WM total volume cm3,Cerebellum WM total volume %,Cerebellum WM right volume cm3,Cerebellum WM right volume %,Cerebellum WM left volume cm3,Cerebellum WM left volume %,Cerebellum WM volume asymmetry,Cerebellum GM total volume cm3,Cerebellum GM total volume %,Cerebellum GM right volume cm3,Cerebellum GM right volume %,Cerebellum GM left volume cm3,Cerebellum GM left volume %,Cerebellum GM volume asymmetry,Vermis volume cm3,Vermis volume %,Brainstem volume cm3,Brainstem volume %,Accumbens total volume cm3,Accumbens total volume %,Accumbens right volume cm3,Accumbens right volume %,Accumbens left volume cm3,Accumbens left volume %,Accumbens volume asymmetry,Amygdala total volume cm3,Amygdala total volume %,Amygdala right volume cm3,Amygdala right volume %,Amygdala left volume cm3,Amygdala left volume %,Amygdala volume asymmetry,Basal Forebrain total volume cm3,Basal Forebrain total volume %,Basal Forebrain right volume cm3,Basal Forebrain right volume %,Basal Forebrain left volume cm3,Basal Forebrain left volume %,Basal Forebrain volume asymmetry,Caudate total volume cm3,Caudate total volume %,Caudate right volume cm3,Caudate right volume %,Caudate left volume cm3,Caudate left volume %,Caudate volume asymmetry,Hippocampus total volume cm3,Hippocampus total volume %,Hippocampus right volume cm3,Hippocampus right volume %,Hippocampus left volume cm3,Hippocampus left volume %,Hippocampus volume asymmetry,Pallidum total volume cm3,Pallidum total volume %,Pallidum right volume cm3,Pallidum right volume %,Pallidum left volume cm3,Pallidum left volume %,Pallidum volume asymmetry,Putamen total volume cm3,Putamen total volume %,Putamen right volume cm3,Putamen right volume %,Putamen left volume cm3,Putamen left volume %,Putamen volume asymmetry,Thalamus total volume cm3,Thalamus total volume %,Thalamus right volume cm3,Thalamus right volume %,Thalamus left volume cm3,Thalamus left volume %,Thalamus volume asymmetry,Ventral DC total volume cm3,Ventral DC total volume %,Ventral DC right volume cm3,Ventral DC right volume %,Ventral DC left vo

## Logistic Regression

In [6]:
# Select the MRI features from the 48th column onwards
mri_features = df.iloc[:, 47:]

# Select the target variable
target_variable = df['NP-SLE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(mri_features, target_variable, test_size=0.2, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an instance of the logistic regression model
logreg = LogisticRegression()

# Fit the model to the training data
logreg.fit(X_train_scaled, y_train)

# Predict the target variable for the test set
y_pred = logreg.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy = logreg.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5


In [7]:
# Select the MRI features from the 48th column onwards
mri_features = df.iloc[:, 47:]

# Select the target variable
target_variable = df['AnAb ']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(mri_features, target_variable, test_size=0.2, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an instance of the logistic regression model
logreg = LogisticRegression()

# Fit the model to the training data
logreg.fit(X_train_scaled, y_train)

# Predict the target variable for the test set
y_pred = logreg.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy = logreg.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.3333333333333333
